In [1]:
from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd
from astropy.time import Time
import time

def makeHorFile(objectName):
    """This function takes a name of a solar system body(a KBO), and creates a csv file of the body's ephemerides"""
    starttime = time.time()
    
    ourKBO = Horizons(id=objectName,location=399,epochs = {'start':'1950-01-01','stop':'2020-01-28','step':'1d'})
    ephKBO = ourKBO.ephemerides()['RA','DEC','datetime_jd']
    vecKBO = ourKBO.vectors()['lighttime']
    
    jdTime= ephKBO['datetime_jd']
    lightTime = vecKBO
    kboTime=jdTime-lightTime
    
    geocentricFile = pd.DataFrame({'geocentricTime':ephKBO['datetime_jd'],'RA':ephKBO['RA'],'DEC':ephKBO['DEC']})
    outFile = pd.DataFrame({'kboTIME':kboTime,'RA':ephKBO['RA'],'DEC':ephKBO['DEC']})
    
    filename1 = objectName+'Data_geoTime.csv'
    geocentricFile.to_csv(filename1)
    fileName2 = objectName+'Data_KBOtime.csv'
    outFile.to_csv(fileName2)
    
    endtime = time.time()
    
    runLength = endtime-starttime
    print("Runtime was: "+str(runLength)+" seconds" )
    
def geotoKBOtime(objectName):
    starttime = time.time()
    theFile = objectName+'Data_geoTime.csv'
    
    oldData = pd.read_csv(theFile)
    date = oldData['Dates']
    
    date1 = date.iloc[0]
    date2 = date.iloc[-1]
    
    jddate1 = Time(date1,format='jd')
    jddate2 = Time(date2,format='jd')
    
    dateList = []
    
    for i in date:
        dateList.append(Time(i,format='jd'))
                        
    theObject = Horizons(id=objectName, location = 399,epochs=dateList)
    vecObj = theObject.vectors()['lighttime']
    
    kboTime=date-vecObj
    
    
    newData = oldData
    newData['Dates'] = kboTime
    
    newData=newData.rename(columns = {'Dates':'datetime_jd'})
    
    outFile = objectName+'Data_KBOTime.csv'
   
    newData.to_csv(outFile)
    
    endtime = time.time()
    
    runLength = endtime-starttime
    print("Runtime was: "+str(runLength)+" seconds" )
    

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [3]:
makeHorFile('Haumea')

Runtime was: 174.04425835609436 seconds
